<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/2013_Contributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **2013 Indivdual Contributions**

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pandas as pd


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
import findspark
findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-14 23:03:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-09-14 23:03:59 (9.92 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
# Start Spark session
#from pyspark.sql import SparkSession
#from pyspark.sql.functions import col
#spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual+Contributions+2013+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Individual+Contributions+2013+(UF).csv"), sep=",", header=True)
df.show()

+--------+--------+-------+--------+------------------+---------+------+--------+------+----------+--------+---------+----------+-------------------+------+-----+-------+---------+---------+-----------------+-----+-----+--------------+--------------------+--------+------------------+---------------+--------+----+---------+--------+----------+--------+----------+--------+--------+--------+-------+-----+------+----------+----------+----------+----------+--------+---------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|     DATE|REFUNDDATE|               NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|             CITY|STATE|  ZIP|    OCCUPATION|             EMPNAME|EMPSTRNO|        EMPSTRNAME|        EMPCITY|EMPSTATE|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO|INTCITY|INTST|INTZIP|INTEMPNAME|INTEMPSTNO|INTEMPSTNM|INTEMPCITY|INTEMPST|INTO

In [7]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','INTERMNO', 'REFUNDDATE', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+-------+--------+------------------+---------+-------------------+------+---------+-----------------+-----+-----+--------+----+---------+--------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|     DATE|               NAME|C_CODE|BOROUGHCD|             CITY|STATE|  ZIP|EMPSTATE|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+-------+--------+------------------+---------+-------------------+------+---------+-----------------+-----+-----+--------+----+---------+--------+----------+
|    2013|       5|   1008|       P|Peterson, Daniel P|3/11/2012|   Haywood, David B|   IND|        X|            Bronx|   NY|10458|      NY|  20|       20|       0|         4|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/6/2012|      Herlihy, Neil|   IND|        Z|       Hunnington|   NY|11746|    null|  25|        0|       0|         2|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/5/2012|       Iocca, Karen|   IND|        Z|           Ho

In [8]:
# check data types
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('RECIPID', 'string'),
 ('CANCLASS', 'string'),
 ('RECIPNAME', 'string'),
 ('DATE', 'string'),
 ('NAME', 'string'),
 ('C_CODE', 'string'),
 ('BOROUGHCD', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('EMPSTATE', 'string'),
 ('AMNT', 'string'),
 ('MATCHAMNT', 'string'),
 ('PREVAMNT', 'string'),
 ('PAY_METHOD', 'string')]

In [9]:
#Change column names
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("RECIPID", "CandidateID") \
.withColumnRenamed("RECIPNAME", "CandidateName") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("C_CODE", "ContributerType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("EMPSTATE", "ContributionState") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod")


df1.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- MatchAmt: string (nullable = true)
 |-- PrevAmt: string (nullable = true)
 |-- PayMethod: string (nullable = true)



In [10]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [11]:
# using lambda function to convert date col to datetype
# changing datatypes of columns
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
df2 = df1.withColumn("Election", df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  .withColumn("MatchAmt", df1["MatchAmt"].cast("Float")) \
  .withColumn("PrevAmt", df1["PrevAmt"].cast("Float")) \
  .withColumn('Date', func(col('Date')))
df2.printSchema()

root
 |-- Election: integer (nullable = true)
 |-- OFFICECD: integer (nullable = true)
 |-- CandidateID: integer (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: float (nullable = true)
 |-- MatchAmt: float (nullable = true)
 |-- PrevAmt: float (nullable = true)
 |-- PayMethod: string (nullable = true)



In [12]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df3 = df2.withColumn('ContributerType', regexp_replace('ContributerType', 'CAN', 'Candidate')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'CORP', 'Corporation')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'EMPO', 'Labor Union')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'FAM', 'Candidate Family')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'IND', 'Individual')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMZ', 'Party Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'ORG', 'Orgainization')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'LLC', 'Limited Liability Company')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMP', 'Political Action Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'SPO', 'Spouse')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'OTHR', 'Other')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMC', 'Candidate Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PART', 'Individual')) \


In [13]:
#Change Payment Method Name
df4=df3.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [14]:
#Change Borough name
from pyspark.sql.functions import regexp_replace
df5 = df4.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \


In [15]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df6 = df5.filter((df5.OFFICECD=="1") & (df5.CANCLASS=="P"))
df6.show()

+--------+--------+-----------+--------+--------------------+----------+-------------------+---------------+-------------+--------------+-----+-------+-----------------+------+--------+-------+-----------+
|Election|OFFICECD|CandidateID|CANCLASS|       CandidateName|      Date|               Name|ContributerType|  BoroughName|          City|State|ZipCode|ContributionState|Amount|MatchAmt|PrevAmt|  PayMethod|
+--------+--------+-----------+--------+--------------------+----------+-------------------+---------------+-------------+--------------+-----+-------+-----------------+------+--------+-------+-----------+
|    2013|       1|       1286|       P|Coenen, Jr., Kevin P|2013-01-25|    Coenen, Kevin P|      Candidate|        Other|     Nesconset|   NY|  11767|               NY|  10.0|     0.0|    0.0|Credit Card|
|    2013|       1|       1286|       P|Coenen, Jr., Kevin P|2013-04-17|     Walter, Dorsey|     Individual|    Manhattan|      New York|   NY|  10150|             null| 175.0|

In [16]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df6.select('OFFICECD').distinct().show()

+--------+
|OFFICECD|
+--------+
|       1|
+--------+



In [17]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df7 = df6.drop(*list2)
df7.show()

+--------+-----------+--------------------+----------+-------------------+---------------+-------------+--------------+-----+-------+-----------------+------+--------+-------+-----------+
|Election|CandidateID|       CandidateName|      Date|               Name|ContributerType|  BoroughName|          City|State|ZipCode|ContributionState|Amount|MatchAmt|PrevAmt|  PayMethod|
+--------+-----------+--------------------+----------+-------------------+---------------+-------------+--------------+-----+-------+-----------------+------+--------+-------+-----------+
|    2013|       1286|Coenen, Jr., Kevin P|2013-01-25|    Coenen, Kevin P|      Candidate|        Other|     Nesconset|   NY|  11767|               NY|  10.0|     0.0|    0.0|Credit Card|
|    2013|       1286|Coenen, Jr., Kevin P|2013-04-17|     Walter, Dorsey|     Individual|    Manhattan|      New York|   NY|  10150|             null| 175.0|   175.0|    0.0|Credit Card|
|    2013|       1286|Coenen, Jr., Kevin P|2013-01-25|    Co

In [18]:
#Filter Dataframe removing the null CandidateID values
df8 = df7.filter(df7.CandidateID.isNotNull())

In [19]:
# ADD AMNT and MATCHAMNT PrevAmnt
from pyspark.sql.functions import col
clean_indiv_2013_df = df8.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
clean_indiv_2013_df.show()

+--------+-----------+--------------------+----------+-------------------+---------------+-------------+--------------+-----+-------+-----------------+------+--------+-------+-----------+-----------+
|Election|CandidateID|       CandidateName|      Date|               Name|ContributerType|  BoroughName|          City|State|ZipCode|ContributionState|Amount|MatchAmt|PrevAmt|  PayMethod|TotalAmount|
+--------+-----------+--------------------+----------+-------------------+---------------+-------------+--------------+-----+-------+-----------------+------+--------+-------+-----------+-----------+
|    2013|       1286|Coenen, Jr., Kevin P|2013-01-25|    Coenen, Kevin P|      Candidate|        Other|     Nesconset|   NY|  11767|               NY|  10.0|     0.0|    0.0|Credit Card|       10.0|
|    2013|       1286|Coenen, Jr., Kevin P|2013-04-17|     Walter, Dorsey|     Individual|    Manhattan|      New York|   NY|  10150|             null| 175.0|   175.0|    0.0|Credit Card|      350.0|


In [20]:
#Export to Clean CSV
#clean_indiv_2013_df.toPandas().to_csv("Clean_Individual_2013.csv", header=True)

## **2013 Committee Contributions**

In [21]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Committee+Contributions+2013+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Committee+Contributions+2013+(UF).csv"), sep=",", header=True)
df.show()

+--------+--------+-------+--------+------------------+---------+------+--------+------+----------+--------+---------+----------+--------------------+------+-----+-------+---------+---------+------------+-----+-----+----------+-------+--------+----------+-------+--------+---------+---------+--------+----------+--------+------------+--------+--------+--------+--------+-----+------+--------------------+----------+-----------+----------+--------+-----------------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|     DATE|REFUNDDATE|                NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|        CITY|STATE|  ZIP|OCCUPATION|EMPNAME|EMPSTRNO|EMPSTRNAME|EMPCITY|EMPSTATE|     AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|  INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO| INTCITY|INTST|INTZIP|          INTEMPNAME|INTEMPSTNO| INTEMPSTNM|INTEMPCITY|INTEMPST|        INTOCCUPA|PURPOSECD|EXEM

In [22]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','REFUNDDATE','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+-------+--------+------------------+---------+--------------------+------+---------+------------+-----+-----+--------+---------+---------+--------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|     DATE|                NAME|C_CODE|BOROUGHCD|        CITY|STATE|  ZIP|EMPSTATE|     AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+-------+--------+------------------+---------+--------------------+------+---------+------------+-----+-----+--------+---------+---------+--------+----------+
|    2013|       1|    204|       P|Quinn, Christine C|7/11/2011|1199 SEIU PAC - N...| PCOMP|        M|    New York|   NY|10036|    null|     1000|        0|       0|         2|
|    2013|       1|    204|       P|Quinn, Christine C|1/10/2013|1199 SEIU PAC - N...| PCOMP|        M|    New York|   NY|10036|    null|     3950|        0|    1000|         2|
|    2013|       1|    326|       P|   de Blasio, Bill|4/29/2010|1199/SEIU NYS Pol...| PCOMP|        M|    New

In [23]:
# check data types
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('RECIPID', 'string'),
 ('CANCLASS', 'string'),
 ('RECIPNAME', 'string'),
 ('DATE', 'string'),
 ('NAME', 'string'),
 ('C_CODE', 'string'),
 ('BOROUGHCD', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('EMPSTATE', 'string'),
 ('AMNT', 'string'),
 ('MATCHAMNT', 'string'),
 ('PREVAMNT', 'string'),
 ('PAY_METHOD', 'string')]

In [24]:
#Change column names
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("RECIPID", "CandidateID") \
.withColumnRenamed("RECIPNAME", "CandidateName") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("C_CODE", "ContributerType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("EMPSTATE", "ContributionState") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod") 

df1.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- MatchAmt: string (nullable = true)
 |-- PrevAmt: string (nullable = true)
 |-- PayMethod: string (nullable = true)



In [25]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [26]:
# using lambda function to convert date col to datetype
# changing datatypes of columns
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
df2 = df1.withColumn("Election",df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  .withColumn("MatchAmt", df1["MatchAmt"].cast("Float")) \
  .withColumn("PrevAmt", df1["PrevAmt"].cast("Float")) \
  .withColumn('Date', func(col('Date')))
df2.printSchema()

root
 |-- Election: integer (nullable = true)
 |-- OFFICECD: integer (nullable = true)
 |-- CandidateID: integer (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: float (nullable = true)
 |-- MatchAmt: float (nullable = true)
 |-- PrevAmt: float (nullable = true)
 |-- PayMethod: string (nullable = true)



In [27]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df3 = df2.withColumn('ContributerType', regexp_replace('ContributerType', 'CAN', 'Candidate')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'CORP', 'Corporation')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'EMPO', 'Labor Union')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'FAM', 'Candidate Family')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'IND', 'Individual')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMZ', 'Party Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'ORG', 'Orgainization')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'LLC', 'Limited Liability Company')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMP', 'Political Action Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'SPO', 'Spouse')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'OTHR', 'Other')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMC', 'Candidate Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PART', 'Individual')) \


In [28]:
#Change Payment Method Name
df4=df3.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [29]:
#Change Borough name
from pyspark.sql.functions import regexp_replace
df5 = df4.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \


In [30]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df6 = df5.filter((df5.OFFICECD=="1") & (df5.CANCLASS=="P"))
df6.show()

+--------+--------+-----------+--------+------------------+----------+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+
|Election|OFFICECD|CandidateID|CANCLASS|     CandidateName|      Date|                Name|     ContributerType|BoroughName|        City|State|ZipCode|ContributionState|   Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----------+--------+------------------+----------+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+
|    2013|       1|        204|       P|Quinn, Christine C|2011-07-11|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   1000.0|     0.0|    0.0|    Check|
|    2013|       1|        204|       P|Quinn, Christine C|2013-01-10|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   

In [31]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df6.select('OFFICECD').distinct().show()

+--------+
|OFFICECD|
+--------+
|       1|
+--------+



In [32]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df7 = df6.drop(*list2)
df7.show()

+--------+-----------+------------------+----------+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+
|Election|CandidateID|     CandidateName|      Date|                Name|     ContributerType|BoroughName|        City|State|ZipCode|ContributionState|   Amount|MatchAmt|PrevAmt|PayMethod|
+--------+-----------+------------------+----------+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+
|    2013|        204|Quinn, Christine C|2011-07-11|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   1000.0|     0.0|    0.0|    Check|
|    2013|        204|Quinn, Christine C|2013-01-10|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   3950.0|     0.0| 1000.0|    Check|
|    2013|        326|   de Blasio, Bill|2010-04-29|119

In [33]:
#Filter Dataframe removing the null CandidateID values
df8 = df7.filter(df7.CandidateID.isNotNull())

In [34]:
# ADD AMNT and MATCHAMNT PrevAmnt
from pyspark.sql.functions import col
clean_comm_2013_df = df8.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
clean_comm_2013_df.show()

+--------+-----------+--------------------+----------+--------------------+--------------------+-----------+----------+-----+-------+-----------------+---------+--------+-------+---------+-----------+
|Election|CandidateID|       CandidateName|      Date|                Name|     ContributerType|BoroughName|      City|State|ZipCode|ContributionState|   Amount|MatchAmt|PrevAmt|PayMethod|TotalAmount|
+--------+-----------+--------------------+----------+--------------------+--------------------+-----------+----------+-----+-------+-----------------+---------+--------+-------+---------+-----------+
|    2013|        204|  Quinn, Christine C|2011-07-11|1199 SEIU PAC - N...|Political Action ...|  Manhattan|  New York|   NY|  10036|             null|   1000.0|     0.0|    0.0|    Check|     1000.0|
|    2013|        204|  Quinn, Christine C|2013-01-10|1199 SEIU PAC - N...|Political Action ...|  Manhattan|  New York|   NY|  10036|             null|   3950.0|     0.0| 1000.0|    Check|     495

In [35]:
# import modules
from pyspark.sql import SparkSession
import functools
  
# explicit function
def unionAll(dfs):
    return functools.reduce(lambda clean_indiv_2013_df, clean_comm_2013_df: clean_indiv_2013_df.union(clean_comm_2013_df.select(clean_indiv_2013_df.columns)), dfs)

unioned_df = unionAll([clean_indiv_2013_df, clean_comm_2013_df])
unioned_df.show()

+--------+-----------+--------------------+----------+-------------------+---------------+-------------+--------------+-----+-------+-----------------+------+--------+-------+-----------+-----------+
|Election|CandidateID|       CandidateName|      Date|               Name|ContributerType|  BoroughName|          City|State|ZipCode|ContributionState|Amount|MatchAmt|PrevAmt|  PayMethod|TotalAmount|
+--------+-----------+--------------------+----------+-------------------+---------------+-------------+--------------+-----+-------+-----------------+------+--------+-------+-----------+-----------+
|    2013|       1286|Coenen, Jr., Kevin P|2013-01-25|    Coenen, Kevin P|      Candidate|        Other|     Nesconset|   NY|  11767|               NY|  10.0|     0.0|    0.0|Credit Card|       10.0|
|    2013|       1286|Coenen, Jr., Kevin P|2013-04-17|     Walter, Dorsey|     Individual|    Manhattan|      New York|   NY|  10150|             null| 175.0|   175.0|    0.0|Credit Card|      350.0|


In [36]:
#Export to new CSV
unioned_df.toPandas().to_csv("Unioned_2013.csv", header=True)

PythonException: ignored

In [ ]:
# Store environmental variable
from getpass import getpass
#Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-final.cjdbapst3wjf.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "*****",
          "driver":"org.postgresql.Driver"}

In [ ]:
#Export to new CSV
#clean_comm_2013_df.toPandas().to_csv("Clean_Committee_2013.csv", header=True)

In [ ]:
## Write DataFrame to active_user table in RDS
#clean_indiv_2013_df.write.jdbc(url=jdbc_url, table='individual_2013', mode=mode, properties=config)

In [ ]:
# Write DataFrame to active_user table in RDS
#clean_comm_2013_df.write.jdbc(url=jdbc_url, table='committee_2013', mode=mode, properties=config)